# Detectron2 (FAIR)


In [ ]:
import detectron2
import contextlib
import datetime
import io
import os
import json
import logging
import cv2
import random
import numpy as np
import copy,torch,torchvision
import PIL
from PIL import Image
import xml.etree.ElementTree as X
import math
from itertools import repeat

from detectron2.structures import Boxes, BoxMode, PolygonMasks
from detectron2.config import *
from detectron2.modeling import build_model
from detectron2 import model_zoo
from detectron2.data import transforms as T
from detectron2.data import detection_utils as utils
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import COCOEvaluator,DatasetEvaluators, inference_on_dataset, coco_evaluation, DatasetEvaluator
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
import matplotlib.pyplot as plt

import glob
import time
import shutil
from multiprocessing.pool import ThreadPool
import concurrent.futures

from utility.LossEvalHook import LossEvalHook
from utility.mytrainer import MyTrainer
from utility.nms import non_max_suppression_fast

import torch
torch.cuda.set_device(0)

setup_logger()

## Dataset preperation

In [ ]:
register_coco_instances("ship_dataset_train", {}, "ShipAnnotations/train2017.json", "ShipDataset")
register_coco_instances("ship_dataset_val", {}, "ShipAnnotations/test2017.json", "ShipDataset")

In [ ]:
register_coco_instances("wake_dataset_train", {}, "Sar Wakes 3/train/coco.json", "Sar Wakes 3/train/imgs")
register_coco_instances("wake_dataset_val", {}, "Sar Wakes 3/validation/coco.json", "Sar Wakes 3/validation/imgs")
# MetadataCatalog.get("wake_dataset_train").set(thing_classes=["wake"])

## Setup configuration
Now we've got out data in a usable form,and some useful functions lets configure our tests. below are the options for training, 

In [ ]:
cfg = get_cfg()
cfg.OUTPUT_DIR = 'output'
#  Let training initialize from models.txt
MODEL = "Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"
cfg.merge_from_file(model_zoo.get_config_file(MODEL))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(MODEL) #
cfg.DATASETS.TRAIN = ("wake_dataset_train",)
cfg.DATASETS.TEST = ("wake_dataset_val",)

# cfg.MODEL.MASK_ON=False
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8 
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # this is far lower than usual.  
cfg.MODEL.ROI_HEADS.NUM_CLASSES =  1             # only has one class.
cfg.MODEL.RETINANET.NUM_CLASSES =  1

cfg.SOLVER.IMS_PER_BATCH = 1          
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.TEST.EVAL_PERIOD = 500
cfg.SOLVER.BASE_LR = 0.005
cfg.SOLVER.GAMMA = 0.5
cfg.SOLVER.STEPS=[1000,1200,2500,3000]
cfg.SOLVER.MAX_ITER=5000

cfg.MODEL.PIXEL_MEAN = [70, 70, 70]
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True 
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)#lets just check our output dir exists

## Training
### Load tensorboard to view progress

In [ ]:
# # Look at training curves in tensorboard:
# %reload_ext tensorboard
# %tensorboard --logdir "output"

### Start training

In [ ]:
# # cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# trainer = MyTrainer(cfg) 
# trainer.resume_or_load(resume=True)
# trainer.train()

## Evaluation

The results produced about aren't as accurate as those performed outside of training. So let's rerun our evaluations. 

In [ ]:
# # Create coco evaluator, but use the default detectron2 data format for generation, make sure ids overlap
# evaluator = COCOEvaluator("wake_dataset_val", cfg, False, output_dir=cfg.OUTPUT_DIR)
# val_loader = build_detection_test_loader(cfg, "wake_dataset_val") 
# outputs = inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
cfg.MODEL.WEIGHTS = "checkpoints/Model Trained Wake/cascade_mask_rcnn_R_50_FPN_3x/model_0002979.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8 
predictor = DefaultPredictor(cfg)

In [ ]:
from utility.myVisualizer import MyVisualizer
# test_dict = get_dataset("wake_dataset_val")
test_img = glob.glob('Sar Wakes 3/validation/imgs/*.png')
for d in random.sample(test_img, 10):
  im = cv2.imread(d)
  outputs = predictor(im)  
  v = MyVisualizer(im[:, :, ::-1],
                  metadata=MetadataCatalog.get("wake_dataset_val"), 
                  scale=1)

 
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  plt.figure(figsize=(10,10))
  plt.imshow(out.get_image()[:, :, ::-1])
  plt.axis(False)
  plt.show()

In [ ]:
# from utility.printProgressBar import printProgressBar

# im = cv2.imread('ScenariTest/ShipWakeTerraSAR-X/VV.tiff')
# y,x = im.shape[0], im.shape[1] 
# step, ws = 500, 650
# range_x, range_y = np.arange(ws, x-ws, step), np.arange(ws, y-ws, step)
# print(f"Number of subsets to be analyzed: {len(range_x)*len(range_y)}")
# Bboxes = []
# i=0
# for space_x in range_x:
#     for space_y in range_y:
#         printProgressBar(i, len(range_x)*len(range_y), printEnd=' ')
#         i+=1
#         img = im[(space_y-ws):(space_y+ws),(space_x-ws):(space_x+ws)]
#         outputs = predictor(img)
#         bboxes=outputs["instances"].pred_boxes.tensor.cpu().numpy()
#         for b in bboxes: 
#             # Each row is (x1, y1, x2, y2).
#             b[0], b[1], b[2], b[3] = b[0]+space_x, b[1]+space_y, b[2]+space_x, b[3]+space_y
#             Bboxes.append(b)

# Bboxes_arr = np.array(Bboxes) 

In [ ]:
# bboxes_draw = non_max_suppression_fast(Bboxes_arr, 0.33)
# for box in bboxes_draw:
#     # represents the top left corner of rectangle
#     start_point =  (box[0], box[1])
#     # represents the bottom right corner of rectangle
#     end_point = (box[2], box[3])
#     color = (255, 0, 0)
#     linethickness = 3
#     image = cv2.rectangle(im, start_point-ws, end_point-ws, color, linethickness)
# cv2.imwrite('ImageTest.png', image) 

In [ ]:
# im = cv2.imread('ScenariTest/ShipWakeTerraSAR-X/VV.tiff')
# test = im[7000+4000:9000+4000,6100:8100]
# outputs = predictor(test)  
# v = Visualizer(test[:, :, ::-1],
#                   metadata=MetadataCatalog.get("wake_dataset_val"), 
#                   scale=1)
#                   # instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
#   # )
# out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# plt.figure(figsize=(10,10))
# plt.imshow(out.get_image()[:, :, ::-1])
# plt.axis(False)
# plt.show()

In [ ]:
# im = cv2.imread('ScenariTest/ShipWakeTerraSAR-X/VV.tiff')
# test = im[7000+4000:9000+4000,6100:8100]
# outputs = predictor(test)  

# bboxes=outputs["instances"].pred_boxes.tensor.cpu().numpy()
# for box in bboxes:
#     start_point =  (box[0], box[1])
#         # represents the bottom right corner of rectangle
#     end_point = (box[2], box[3])
#     color = (255, 0, 0)
#     linethickness = 5
#     image = cv2.rectangle(test, start_point, end_point, color, linethickness)
# cv2.imwrite('ImageTest.png', image) 

In [ ]:
bboxes